In [1]:
import numpy as np
import pandas as pd
import scanpy as sc
import anndata as ad
import squidpy as sq
import json
import muon as mu
import cellcharter as cc
import json
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import glasbey
from tol_colors import tol
import seaborn as sns

/Users/lukashat/miniforge3/envs/cellcharter_scarches/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
 captum (see https://github.com/pytorch/captum).


ModuleNotFoundError: No module named 'glasbey'

# Using CLR form muon

In [3]:
adata = ad.read_h5ad("/Users/lukashat/Documents/PhD_Schapiro/Projects/Myeloma_Standal/results/standard/adatas/cells_final.h5ad")

In [17]:
adata.X = adata.layers['arcsinh']

In [ ]:
mu.prot.pp.clr(adata, axis=1) #per cell normalization for intra-celltype comparisons

In [ ]:
adata_clr.X

In [ ]:
adata_clr.write_h5ad("/Users/lukashat/Documents/PhD_Schapiro/Projects/Myeloma_Standal/results/standard/adatas/cells_current_clr_sample.h5ad")

# Preprocess

In [ ]:
adata = ad.read_h5ad("/Users/lukashat/Documents/PhD_Schapiro/Projects/Myeloma_Standal/results/standard/adatas/cells_current_clr_sample.h5ad")
adata.obsm['spatial'] = np.array(adata.obs[['X_centroid', 'Y_centroid']])
adata.uns['spatial'] = {
    'X': adata.obs['X_centroid'].values,
    'Y': adata.obs['Y_centroid'].values
}
unique_image_ids = adata.obs['image_ID'].unique()
adata.uns['spatial'] = {image_id: {} for image_id in unique_image_ids}
sq.gr.spatial_neighbors(adata, library_key='image_ID', coord_type='generic', delaunay=True)
cc.gr.remove_long_links(adata, distance_percentile=95)
adata.obs['disease3'] = adata.obs['disease2']
adata.obs['disease3'] = adata.obs['disease3'].astype(str)
adata.obs['disease3'] = adata.obs['disease3'].replace(['MM_BD', 'MM_noBD'], 'MM')
adata.obs['disease3'] = adata.obs['disease3'].astype('category')


In [ ]:
adata.write_h5ad("/Users/lukashat/Documents/PhD_Schapiro/Projects/Myeloma_Standal/results/standard/adatas/cells_current_clr_sample_pp.h5ad")

# Final celltype nad NBH renaming

In [ ]:
adata = ad.read_h5ad("/Users/lukashat/Documents/PhD_Schapiro/Projects/Myeloma_Standal/results/standard/adatas/cells_current_clr_sample_pp.h5ad")

In [ ]:
adata.obs['Phenotype4'].value_counts()

In [ ]:
adata.obs['Phenotype4'] = adata.obs['Phenotype4'].astype(str)
adata.obs['Phenotype4'] = adata.obs['Phenotype4'].replace({'CD4+Tcells': 'CD4 T', 'CD8+Tcells': 'CD8 T', 'Dendritic Cells': 'DCs', 'Endothelial cells': 'Endothelial',
                                                           'HSCs': 'HSPCs', 'Macrophages/Monocytes': 'Macs/monos', 'activated Macrophages/Monocytes': 'HLA-DR+ Macs/monos',
                                                           'Osteoblast lineage': 'OB/RUNX2+', 'Osteocyte': 'Osteocytes', 'Plasma Cells/MM cells': 'PCs'})
adata.obs['Phenotype4'] = adata.obs['Phenotype4'].astype('category')
adata.obs['Phenotype4'].value_counts()

In [ ]:
adata.obs['cellcharter_CN'].value_counts()

In [ ]:
adata.obs['cellcharter_CN'] = adata.obs['cellcharter_CN'].astype(str)
adata.obs['cellcharter_CN'] = adata.obs['cellcharter_CN'].replace({'pc_myeloid':'PC_MYELOID', 'hypoxic_immune': 'HIF1A_IMMUNE', 'proliferating_glycolytic': 'PROLIF_GLYC',
                                                                   'focal_pc_oxphos': 'PC_OXPHOS', 'stroma_adipocyte': 'ADIPOCYTES_PC', 'bone_myeloid': 'BONE_MYELOID',
                                                                   'bone_vasculature': 'BONE_VASCULATURE','adaptive_immune': 'ADAPTIVE_IMMUNE', 'bone': 'BONE_ADIPOCYTES'})
adata.obs['cellcharter_CN'] = adata.obs['cellcharter_CN'].astype('category')
adata.obs['cellcharter_CN'].value_counts()

In [ ]:
adata.obs.drop(['Phenotype', 'Phenotype2', 'Phenotype3'], axis=1, inplace=True)

In [ ]:
adata.write_h5ad("/Users/lukashat/Documents/PhD_Schapiro/Projects/Myeloma_Standal/results/standard/adatas/cells_final.h5ad")

In [ ]:
adata.obs